# REVERB

In [1]:
sentences = {}
with open('sentences.txt','r') as f:
    for line in f:
        k,v = line.strip('\n').split('\t')
        sentences[int(k)] = v

In [7]:
sentences[0]

"The nation 's health maintenance organizations were required to tell the federal government by midnight Monday whether they plan to continue providing health insurance to Medicare recipients next year , raise premiums , or reduce benefits ."

In [11]:
from collections import defaultdict
data = {}
data['reverb'] = {}
data['reverb']['labeled'] = defaultdict(dict)

In [12]:
with open('reverb.txt','r') as f:
    for line in f:
        sid,arg1,rel,arg2,confidence = line.strip('\n').split('\t')
        sid = int(sid)
        confidence = float(confidence)
        data['reverb']['labeled'][sid]['sentence'] = sentences[sid]
        if 'extractions' in data['reverb']['labeled'][sid]:
            data['reverb']['labeled'][sid]['extractions'].append({'extracted_triple':(arg1,rel,arg2),
                                                                  'confidence': confidence
                                                                  })
        else:
            data['reverb']['labeled'][sid]['extractions'] = [{'extracted_triple':(arg1,rel,arg2),
                                                              'confidence': confidence
                                                              }]

In [14]:
from pprint import pprint
pprint(data['reverb']['labeled'][0])

{'extractions': [{'confidence': 0.490821038033,
                  'extracted_triple': ("The nation 's health maintenance "
                                       'organizations',
                                       'were required to tell',
                                       'the federal government')},
                 {'confidence': 0.327266866433,
                  'extracted_triple': ('Medicare recipients',
                                       'raise',
                                       'premiums')},
                 {'confidence': 0.259293275069,
                  'extracted_triple': ('premiums', 'reduce', 'benefits')}],
 'sentence': "The nation 's health maintenance organizations were required "
             'to tell the federal government by midnight Monday whether they '
             'plan to continue providing health insurance to Medicare '
             'recipients next year , raise premiums , or reduce benefits .'}


In [24]:
#In case of emergency, uncomment:
#for sid in data['reverb']['labeled'].keys():
#    data['reverb']['labeled'][sid]['labels'] = []
with open('labels.txt','r') as f:
    for line in f:
        label,sid,arg1,rel,arg2 = line.strip('\n').split('\t')
        sid = int(sid)
        label = True if int(label) else False
        if 'labels' in data['reverb']['labeled'][sid]:
            data['reverb']['labeled'][sid]['labels'].append({ 'is_correct' : label,
                                                              'triple' : (arg1,rel,arg2)
                                                            })
        else:
            data['reverb']['labeled'][sid]['labels'] = [{ 'is_correct' : label,
                                                          'triple' : (arg1,rel,arg2)
                                                        }]

In [25]:
pprint(data['reverb']['labeled'][0]['labels'])

[{'is_correct': True,
  'triple': ('they', 'plan to continue providing', 'health insurance')},
 {'is_correct': False, 'triple': ('premiums', 'or reduce', 'benefits')},
 {'is_correct': False, 'triple': ('they', 'plan raise', 'premiums')},
 {'is_correct': False,
  'triple': ("The nation 's health maintenance organizations",
             'were required to tell the federal government by',
             'midnight Monday')},
 {'is_correct': False, 'triple': ('they', 'plan reduce', 'benefits')},
 {'is_correct': False, 'triple': ('Medicare recipients', 'raise', 'premiums')},
 {'is_correct': False, 'triple': ('next year', 'raise', 'premiums')},
 {'is_correct': False, 'triple': ('premiums', 'reduce', 'benefits')},
 {'is_correct': False,
  'triple': ('they', 'plan to continue providing', 'next year')}]


In [28]:
import pickle
pickle.dump(data,open('temp-commit-reverb-works.pkl','wb'))

In [39]:
data['reverb']['labeled'][22]

{'labels': [{'is_correct': False,
   'triple': ('Esaka', 'will quit to take', 'responsibility')},
  {'is_correct': False,
   'triple': ('six other top executives',
    'will quit to take',
    'responsibility')}]}

In [43]:
#Strangely, there are sentences for which no extraction was provided
#They are printed in this block
print("SENTENCES WITHOUT ANY EXTRACTIONS:\n\n")
scores = []
for sid in data['reverb']['labeled'].keys():
    sumconf = 0
    n = 0
    try:
        for e in data['reverb']['labeled'][sid]['extractions']:
            sumconf += e['confidence']
            n += 1
        scores.append(((sumconf/n),sentences[sid]))
    except KeyError:
        print(sentences[sid])

SENTENCES WITHOUT ANY EXTRACTIONS:


Esaka and six other top executives will quit to take responsibility for 67.28 million yen in payoffs to corporate racketeer Ryuichi Koike , 54 .
He died en route to hospital .
In previous games , Edwards believes that defenders were pressing , trying to overcompensate for the lack of certainty about the system by leaving their assignments to attempt a big play .
It did n't specify which goods in either case .
The committee is planning to hear from him this month .
The first photograph of the fatal crash in which Diana , Princess of Wales , died appeared on the front page of Bild Zeitung Monday .
Tropical Storm Edouard will meander about 125 miles off the Georgia coast causing surf to build along parts of the Southeast Coast .
The first batter he faced , Carl Everett , singled .
If his use of the Web to create art sounds highfalutin , Koplowitz begs to differ .
The p53 protein is known to work as a genetic switch , binding to various sites along the 

In [49]:
#20 Reverb sentences with low confidence score extractions
for conf,sent in sorted(scores)[:20]:
    print(conf,sent)

0.17824928624 Moreover , he said that foreign students spend an estimated $ 11 billion a year for tuition and rent , not to mention items ranging from jeans to pizzas .
0.17824928624 Next up on the politics-and-calories calendar is a Buell dinner Monday for Chellie Pingree , the Dems ' Maine senatorial candidate .
0.17824928624 The group 's attacks on the Kurdish leadership give it common cause with Saddam Hussein , though a firm link with the Iraqi dictator has not been established .
0.182964838443 An online bidder in Boerne paid $ 525 for a 20-inch Sony television set that retails at less than $ 350 new .
0.198098173729 A year ago , three-fifths of Americans said the government had done enough to protect them against another terrorist attack ; now just two-fifths do .
0.198098173729 And as city officials scrambled to recruit new foster parents , they found themselves struggling to find homes for teen-agers , children suffering from psychological disorders , young mothers with babies 

# OLLIE

##### Unlike Reverb, results/data on Ollie don't seem well documented
##### What follows is only my interpretation. I might be wrong.
*Assumption: format of ollie-scored.txt is correct?,confidence,extracted_triple,sentence*

In [58]:
data['ollie'] = {}
data['ollie']['labeled'] = defaultdict(dict)

In [59]:
sentences_ollie = {} #no predefined id
sentences_reverse_dict = {}
sid = 0
with open('ollie-scored.txt','r') as f:
    for line in f:
        label,confidence,triple,sentence = line.strip('\n').split('\t')
        label = True if int(label) else False
        confidence = float(confidence)
        triple = tuple(triple[1:-1].split('; '))   #for consistency with reverb above
        if sentence not in sentences_reverse_dict:
            sid += 1
            sentences_reverse_dict[sentence] = sid
            sentences_ollie[sid] = sentence
            data['ollie']['labeled'][sid]['sentence'] = sentence
        else:
            sid = sentences_reverse_dict[sentence]
        if 'extractions' in data['ollie']['labeled'][sid]:
            data['ollie']['labeled'][sid]['extractions'].append({ 'is_correct' : label,
                                                              'extracted_triple' : triple,
                                                              'confidence': confidence
                                                            })
        else:
            data['ollie']['labeled'][sid]['extractions'] = [{ 'is_correct' : label,
                                                          'triple' : triple,
                                                          'confidence': confidence
                                                        }]

In [60]:

scores_ollie_inc = []

scores_ollie_cor = []

for sid in data['ollie']['labeled'].keys():
    for e in data['ollie']['labeled'][sid]['extractions']:
        if e['is_correct']:
            scores_ollie_cor.append((e['confidence'],data['ollie']['labeled'][sid]['sentence']))
        else:
            scores_ollie_inc.append((e['confidence'],data['ollie']['labeled'][sid]['sentence']))
     
    
s1 = sorted(scores_ollie_cor)[:10]
s2 = sorted(scores_ollie_inc,reverse=True)[:10]

print("#10 low confidence correct\n")
pprint(s1)
print()
print()
print("#10 high confidence incorrect\n")
pprint(s2)

#10 low confidence correct

[(0.039389305142903994,
  'Among professionals , 76 % have a favorable opinion of her , compared to '
  "62 % who approve of her husband 's performance ."),
 (0.0676623135388099,
  'Under the proposal by Equitec , a financially troubled real-estate '
  'syndicator , New York-based Hallwood Group Inc. would replace Equitec as '
  "the newly formed master limited partnership 's general partner and "
  'manager .'),
 (0.09296550914554585,
  "Among nonsmokers , birth control pills slightly raise a woman 's risk of "
  'abnormal blood clotting , high blood pressure , heart attack , and '
  'stroke .'),
 (0.09553548327091532,
  'Among professionals , 76 % have a favorable opinion of her , compared to '
  "62 % who approve of her husband 's performance ."),
 (0.14930191032412005,
  'Among professionals , 76 % have a favorable opinion of her , compared to '
  "62 % who approve of her husband 's performance ."),
 (0.18898511201815474,
  "It is the presence of interna

In [62]:
for score,sent in s1:
    print(sent)
    
print("\n\n")
    
for score,sent in s2:
    print(sent)

Among professionals , 76 % have a favorable opinion of her , compared to 62 % who approve of her husband 's performance .
Under the proposal by Equitec , a financially troubled real-estate syndicator , New York-based Hallwood Group Inc. would replace Equitec as the newly formed master limited partnership 's general partner and manager .
Among nonsmokers , birth control pills slightly raise a woman 's risk of abnormal blood clotting , high blood pressure , heart attack , and stroke .
Among professionals , 76 % have a favorable opinion of her , compared to 62 % who approve of her husband 's performance .
Among professionals , 76 % have a favorable opinion of her , compared to 62 % who approve of her husband 's performance .
It is the presence of internal and external `` enemies '' which justifies the need for a large , active army that Mikhail Gorbachev 's Soviet Union continues to supply with bullets .
At the Southern Commercial Convention of 1854 , Pike said the South should remain in 

In [67]:
#Transaction point. Save data dict to file.
pickle.dump(data,open('temp-commit-reverb-and-ollie-work.pkl','wb'))

## WOE
While the release / web page of WOE has a number of annotated data releases, there isn't (to my knowledge) test data that been annotated as wrongly or rightly extracted. This is data that was released with Reverb, which analysed what kind of properties the relations extracted by WOE.  
*format: sent relation contig_matches POS_between_args meets_constraints reason*

In [63]:
data['woe'] = {}
data['woe']['labeled'] = defaultdict(dict)

In [69]:
with open('woe_parse.txt','r') as f:
    for line in f:
        sid,arg1,rel,arg2,confidence = line.strip('\n').split('\t')
        sid = int(sid)
        confidence = float(confidence)
        data['woe']['labeled'][sid]['sentence'] = sentences[sid]
        if 'extractions' in data['woe']['labeled'][sid]:
            data['woe']['labeled'][sid]['extractions'].append({'extracted_triple':(arg1,rel,arg2),
                                                                  'confidence': confidence
                                                                  })
        else:
            data['woe']['labeled'][sid]['extractions'] = [{'extracted_triple':(arg1,rel,arg2),
                                                              'confidence': confidence
                                                              }]

In [72]:
scores_woe = []
for sid in data['woe']['labeled'].keys():
    sumconf = 0
    n = 0
    try:
        for e in data['woe']['labeled'][sid]['extractions']:
            sumconf += e['confidence']
            n += 1
        scores_woe.append(((sumconf/n),sentences[sid]))
    except KeyError:
        print(sentences[sid])

In [74]:
for conf,sent in sorted(scores_woe)[:20]:
    print(sent)

She was a little anxious early on .
The other five are still alive and will be allowed to return home if they want .
The yield spread between an index of option-free 10-year `` A2 '' bank bonds and the 10-year Treasury note was 70 basis points yesterday , 2 basis points wider from a week earlier .
The Rams sorely miss departed third receiver Az Hakim .
The board of directors of UAL , the parent company of United Airlines , is likely to name Glenn F. Tilton , an oil industry executive , as the airline 's new president and chief executive in a meeting Monday morning , according to people with knowledge of the situation .
The rules were written to encourage diversity of voices on the airwaves and competition among media outlets , and to prevent the biggest companies from becoming too powerful in controlling news and entertainment .
There should be about 3 cups vegetables chopped medium fine .
Some think the blend is more weighted toward expedience .
They litter the road to redemption with

### Processing Reverb results to make it look like the others

In [2]:
reverb_results = {}
with open('results-cleaner-reverb+arglearner.txt') as f:
    for line in f:
        sid,arg1,rel,arg2,conf,sent = line.strip('\n').split('\t')
        sid = int(sid)
        conf = float(conf)
        if sid in reverb_results:
            reverb_results[sid]['extractions'].append({
                                                        'confidence':conf,
                                                        'triple':(arg1,rel,arg2)
                                                    })
        else:
            reverb_results[sid] = { 'sentence':sent }
            reverb_results[sid]['extractions'] = [{
                                                        'confidence':conf,
                                                        'triple':(arg1,rel,arg2)
                                                    }]

In [7]:
for sid in reverb_results.keys():
    print(reverb_results[sid]['sentence'])
    for e in reverb_results[sid]['extractions']:
        print("%.3f: (%s; %s; %s)"%(e['confidence'],e['triple'][0],e['triple'][1],e['triple'][2]))
    print()

Bananas are an excellent source of potassium .
0.951: (Bananas; are an excellent source of; potassium)

There should be about 3 cups vegetables chopped medium fine .
0.461: (about 3 cups vegetables; chopped; medium fine)

Politicians , historians , locals and the victims ' families are grappling with how the site should be preserved .
0.720: (Politicians; are grappling with; how the site should be preserved)
0.737: (historians; are grappling with; how the site should be preserved)
0.753: (locals; are grappling with; how the site should be preserved)
0.669: (the victims ' families; are grappling with; how the site should be preserved)

At least that was the case for linebacker Brendon Ayanbadejo at the Rose Bowl on Saturday .
0.557: (At least that; was the case for; linebacker Brendon Ayanbadejo at the Rose Bowl on Saturday)

Hydra , of Cambridge , Mass. , is named after a pond-dwelling creature that can grow two wholes after being sliced in half .
0.078: (Hydra , of Cambridge , Mass.; 

## Processing Stanford output to put test sentences between the results

In [8]:
test_sentences = []

with open('Common-Input-Some-Preprocessing.txt') as f:
    for line in f:
        test_sentences.append(line.strip('\n'))
i = 0

print(test_sentences[i])

with open('results-stanfordOpenIE.txt') as f:
    for line in f:
        if line == '\n':
            print()
            i += 1
            print(test_sentences[i])
        else:
            print(line.strip('\n'))

Bananas are an excellent source of potassium.
1.000: (Bananas; are source of; potassium)
1.000: (Bananas; are excellent source of; potassium)
1.000: (Bananas; are; excellent)
1.000: (Bananas; are; source)
1.000: (Bananas; are; excellent source)

Esaka and six other top executives will quit to take responsibility for 67.28 million yen in payoffs to corporate racketeer Ryuichi Koike, 54.
1.000: (Esaka; quit; six other top executives)
1.000: (Esaka; quit; other top executives)
1.000: (Esaka; take; six other top executives)
1.000: (Esaka; take; other top executives)

He died en route to hospital.
1.000: (He; died en; route)
1.000: (He; died en; route to hospital)

In previous games, Edwards believes that defenders were pressing, trying to overcompensate for the lack of certainty about the system by leaving their assignments to attempt a big play.
1.000: (Edwards; believes In; previous games)
1.000: (Edwards; believes In; games)

It didn't specify which goods in either case.

The committee 